In [4]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import cv2
import matplotlib.pyplot as plt

from itertools import combinations

import torch.optim as optim

from sklearn.ensemble import VotingClassifier # 보팅 앙상블 클래스

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [8]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [179]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../datasets_fin/'

# model config
model_name = 'resnet50' # 'resnet50' 'efficientnet-b0', ...

# training config
# img_size = 256*2
img_size = 256
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 16
# BATCH_SIZE = 32
num_workers = 0

In [180]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [181]:
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # A.CenterCrop( 224 , 224 , always_apply = False , p = 1.0 ),
    # A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
    ToTensorV2()
])
transform_list = [A.CoarseDropout(max_holes=30, max_height=16, max_width=16, min_holes=10, min_height=16, min_width=16, p=1),
                  # A.Rotate(limit=(45, 45), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(90, 90), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(135, 135), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(180, 180), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(225, 225), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(270, 270), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  # A.Rotate(limit=(315, 315), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  A.Rotate(limit=(0, 360), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
                  A.VerticalFlip(always_apply=False, p=1),
                  A.HorizontalFlip(always_apply=False, p=1),
                #   A.Blur(always_apply=True, p=1, blur_limit=(3, 3)),
                  A.GaussNoise(always_apply=False, p=1, var_limit=(0.2, 0.2)),
                  A.Downscale(always_apply=False, p=1, scale_min=0.5, scale_max=0.5, interpolation=0),
                  A.GaussianBlur(blur_limit=(3, 7), p=1.0),
                  A.RandomGridShuffle( grid = (3 , 3) , always_apply = False , p = 1 ),
                  A.RandomBrightnessContrast( brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True, always_apply=None, p=1 ),
                  # A.GaussianBlur( blur_limit = (3,7) , always_apply = False , p = 1 ),
                  A.ToGray( always_apply = False , p = 1 ),
                #   A.CenterCrop( 224 , 224 , always_apply = False , p = 1.0 ),
                #   A.RandomCrop(224, 224, always_apply=False, p=1.0)
                  ]

tst_transform = A.Compose([ 
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
    ToTensorV2()
])

In [182]:
# Dataset 정의를 위한 함수
def make_dataset(transform_list, csv, path):
    trn_dataset = ImageDataset(
        csv,
        path,
        # transform=transform_list[0]
        transform = trn_transform
    )
    for transform in transform_list:
        trn_dataset2 = ImageDataset(
            csv,
            path,
            transform=A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transform,
                # A.RandomCrop(224, 224, always_apply=False, p=1.0),
                # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                # A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
                ToTensorV2()
            ])
        )
        trn_dataset = ConcatDataset([trn_dataset, trn_dataset2])

    for transform1, transform2 in combinations(transform_list, 2):
        trn_dataset2 = ImageDataset(
            csv,
            path,
            transform=A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transform1,
                transform2,
                # A.RandomCrop(224, 224, always_apply=False, p=1.0),
                # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                # A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
                ToTensorV2()
            ])
        )
        trn_dataset = ConcatDataset([trn_dataset, trn_dataset2])


    return trn_dataset

In [161]:
trn_dataset = make_dataset(transform_list, "../datasets_fin/divided_train.csv", "../datasets_fin/train/")
vaild_dataset = make_dataset(transform_list, "../datasets_fin/vaild.csv", "../datasets_fin/train/",)
tst_dataset = ImageDataset(
    "../datasets_fin/test.csv",
    "../datasets_fin/train/",
    transform=tst_transform
)
print(len(trn_dataset), len(vaild_dataset), len(tst_dataset))

70336 8792 157


In [152]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
vaild_loader = DataLoader(
    vaild_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [121]:
# test 데이터 확인
tst_preds_list = []
tst_target_list = []

model_15.eval()
for image, target in tqdm(tst_loader):
    image = image.float().to(device)

    with torch.no_grad():
        preds = model_15(image)
    tst_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    tst_target_list.extend(target.detach().cpu().numpy())
tst_acc = accuracy_score(tst_target_list, tst_preds_list)
tst_f1 = f1_score(tst_target_list, tst_preds_list, average='macro')

tst_acc, tst_f1

100%|██████████| 5/5 [00:01<00:00,  4.18it/s]


(0.9490445859872612, 0.9452380952380953)

In [16]:
def data_test(model, tst_loader):
    tst_preds_list = []
    tst_target_list = []

    model.eval()
    for image, target in tqdm(tst_loader):
        image = image.float().to(device)

        with torch.no_grad():
            preds = model(image)
        tst_preds_list.extend(preds.detach().cpu().numpy())
        tst_target_list.extend(target.detach().cpu().numpy())
    

    return tst_preds_list, tst_target_list

In [105]:
# 예측 성능 90% 이상의 soft voting
# 이미지 256, 배치 16
model_21 = torch.load('../model/best_augmentation21_256_17.pt')
pred_21, target_21 = data_test(model_21, tst_loader)

100%|██████████| 10/10 [00:01<00:00,  5.24it/s]


In [113]:
# 이미지 512, 배치 32
model_18 = torch.load('../model/best_augmentation18_512_23.pt')
pred_18, target_18 = data_test(model_18, tst_loader)

100%|██████████| 5/5 [00:01<00:00,  2.85it/s]


In [122]:
# 이미지 256, 배치 32
model_17 = torch.load('../model/best_augmentation17_256_21.pt')
pred_17, target_17 = data_test(model_17, tst_loader)

100%|██████████| 5/5 [00:01<00:00,  4.15it/s]


In [124]:
# 이미지 사이즈 256 배치 32
model_15 = torch.load('../model/best_augmentation15_256_6.pt')
pred_15, target_15 = data_test(model_15, tst_loader)

100%|██████████| 5/5 [00:01<00:00,  4.23it/s]


In [141]:
tmp_pred = []
for a1, a2, a3, a4 in zip(pred_21, pred_18, pred_17, pred_15):
    tmp_pred.append(((a1+a2+a3+a4)/4).argmax())

In [142]:
f1_score(target_21, tmp_pred, average='macro')

0.9566333779888767

In [143]:
a = pd.DataFrame({
    'a' : tmp_pred,
    'b' : target_21
})

In [144]:
a

,a,b
0,15,15
1,10,10
2,5,5
3,16,16
4,11,11
...,...,...
152,16,16
153,4,4
154,9,9
155,8,8


In [146]:
a[a['a'] != a['b']]

,a,b
29,7,3
36,3,7
68,3,7
97,4,14
98,3,14
99,3,7


실재 데이터 예측

In [183]:
# 실제 test데이터
test_dataset = ImageDataset(
    "../datasets_fin/sample_submission.csv",
    "../datasets_fin/test/",
    transform=tst_transform
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [184]:
def real_test(model, test_loader):
    preds_list = []

    model.eval()
    for image, _ in tqdm(test_loader):
        image = image.to(device)

        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.detach().cpu().numpy())
    return preds_list

In [185]:
# 예측 성능 90% 이상의 soft voting
# 이미지 256, 배치 16
model_21 = torch.load('../model/best_augmentation21_256_17.pt')
pred_21 = real_test(model_21, test_loader)

  0%|          | 0/197 [00:00<?, ?it/s]

100%|██████████| 197/197 [00:38<00:00,  5.08it/s]


In [171]:
# 이미지 512, 배치 32
model_18 = torch.load('../model/best_augmentation18_512_23.pt')
pred_18 = real_test(model_18, test_loader)

100%|██████████| 99/99 [00:34<00:00,  2.86it/s]


In [177]:
# 이미지 256, 배치 32
model_17 = torch.load('../model/best_augmentation17_256_21.pt')
pred_17 = real_test(model_17, test_loader)

100%|██████████| 99/99 [00:26<00:00,  3.74it/s]


In [178]:
# 이미지 사이즈 256 배치 32
model_15 = torch.load('../model/best_augmentation15_256_6.pt')
pred_15 = real_test(model_15, test_loader)

100%|██████████| 99/99 [00:27<00:00,  3.62it/s]


In [186]:
tmp_pred = []
for a1, a2, a3, a4 in zip(pred_21, pred_18, pred_17, pred_15):
    tmp_pred.append(((a1+a2+a3+a4)/4).argmax())

In [187]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = tmp_pred

In [188]:
sample_submission_df = pd.read_csv("../datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [189]:
pred_df.to_csv("../output/soft_voting_upper90.csv", index=False)

In [190]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
